In [1]:
import pandas as pd
import numpy as np
#coding=utf-8
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn import preprocessing  
min_max_scaler = preprocessing.MinMaxScaler()  
from sklearn.model_selection import KFold,StratifiedKFold

In [59]:
# fulldata = pd.read_csv("../dealinput/addr_recive_CountUnonehot.csv")
# del fulldata["id.1"]

# fulldata.to_csv("../dealinput/addr_recive_CountUnonehot.csv",index = False)  
# fulldata

In [2]:
fulldata = pd.read_csv("../dealinput/addr_recive_CountUnonehot.csv")  
train11 = pd.read_csv("../dealinput/train51ok.csv")
test11 = pd.read_csv("../dealinput/test51ok.csv")

print(train11.shape,test11.shape)
print(train11.isnull().sum().sort_values(ascending = False)[0:5],test11.isnull().sum().sort_values(ascending = False)[0:5])
fulldata

d:\gongju\python3.6 64\anzhuang\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(120929, 194) (47031, 193)
orderInfo_buy_money_maxCutave    0
phoneTop3_182                    0
bank_name_more_cate_n            0
bank_name_more_cate_l            0
bank_name_more_cate_k            0
dtype: int64 orderInfo_buy_money_maxCutave    0
phone_more_cate_top3_159         0
bank_name_more_cate_n            0
bank_name_more_cate_l            0
bank_name_more_cate_k            0
dtype: int64


,id,addr_more_region_pro,addr_more_region_diji,addr_more_region_city,addr_more_region_proportion,addr_more_Latitude,addr_more_longitude,addr_dis
0,616266486771618056,四川,1,四川资阳市,1.000000,30.796354,104.346430,0.000000e+00
1,616411495579783176,河北,1,河北保定市,0.750000,23.310057,113.818621,2.797932e+06
2,616483294195749128,山东,2,山东临沂市,1.000000,28.447361,119.849952,0.000000e+00
3,616547037910929672,广东,2,广东中山市,1.000000,31.179870,120.365776,0.000000e+00
4,616550964970656008,云南,2,云南昆明市,0.500000,29.050977,111.671999,8.394956e+05
5,616657485251481864,江苏,2,江苏扬州市,0.750000,34.709004,113.721861,8.738135e+05
6,616758963400085768,广西,2,广西北海市,1.000000,30.856143,120.744588,0.000000e+00
7,616772308744278024,江苏,1,江苏苏州市,1.000000,36.192897,120.431146,0.000000e+00
8,616777971096621320,广东,2,广东深圳市,1.000000,32.212549,119.922710,0.000000e+00
9,616782166558248968,四川,2,四川自贡市,1.000000,22.707433,113.930013,0.000000e+00


In [5]:
fulldata.columns

Index(['id', 'addr_more_region_pro', 'addr_more_region_diji',
       'addr_more_region_city', 'addr_more_region_proportion',
       'addr_more_Latitude', 'addr_more_longitude', 'addr_dis'],
      dtype='object')

In [3]:
cate_feature = ['addr_more_region_pro', 'addr_more_region_diji',
                'addr_more_region_city']
num_feature = ['addr_more_region_proportion',
               'addr_more_Latitude', 'addr_more_longitude', 'addr_dis']

from sklearn import preprocessing  
min_max_scaler = preprocessing.StandardScaler()

for i in range(len(cate_feature)):
    tmp = pd.get_dummies(fulldata[cate_feature[i]],prefix = cate_feature[i])
    fulldata = pd.concat([fulldata,tmp],axis = 1) 
    del fulldata[cate_feature[i]]
num = pd.DataFrame(min_max_scaler.fit_transform(fulldata[num_feature]),columns = num_feature)
fulldata.drop(columns = num_feature,inplace = True)
fulldata = pd.concat([fulldata,num],axis = 1) 
fulldata

,id,addr_more_region_pro_上海,addr_more_region_pro_云南,addr_more_region_pro_内蒙,addr_more_region_pro_北京,addr_more_region_pro_台湾,addr_more_region_pro_吉林,addr_more_region_pro_四川,addr_more_region_pro_天津,addr_more_region_pro_宁夏,...,addr_more_region_city_黑龙江牡丹,addr_more_region_city_黑龙江绥化,addr_more_region_city_黑龙江鸡西,addr_more_region_city_黑龙江鹤岗,addr_more_region_city_黑龙江黑河,addr_more_region_city_黑龙江齐齐,addr_more_region_proportion,addr_more_Latitude,addr_more_longitude,addr_dis
0,616266486771618056,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.540364,-0.094379,-1.999670,-0.478896
1,616411495579783176,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-0.557828,-1.508902,-0.360377,4.487118
2,616483294195749128,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,-0.538217,0.683429,-0.478896
3,616547037910929672,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,-0.021914,0.772699,-0.478896
4,616550964970656008,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-1.656019,-0.424165,-0.731879,1.011114
5,616657485251481864,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-0.557828,0.644910,-0.377122,1.072025
6,616758963400085768,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,-0.083082,0.838258,-0.478896
7,616772308744278024,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,0.925289,0.784012,-0.478896
8,616777971096621320,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,0.173209,0.696020,-0.478896
9,616782166558248968,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.540364,-1.622767,-0.341099,-0.478896


In [4]:
mm = train11[["id","target"]]
fulldata.id = fulldata.id.astype("str")
mm.id = mm.id.astype("str")
train = mm.merge(fulldata,how = 'left',on = 'id')
print(train.isnull().sum())
train

d:\gongju\python3.6 64\anzhuang\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


id                             0
target                         0
addr_more_region_pro_上海        0
addr_more_region_pro_云南        0
addr_more_region_pro_内蒙        0
addr_more_region_pro_北京        0
addr_more_region_pro_台湾        0
addr_more_region_pro_吉林        0
addr_more_region_pro_四川        0
addr_more_region_pro_天津        0
addr_more_region_pro_宁夏        0
addr_more_region_pro_安徽        0
addr_more_region_pro_山东        0
addr_more_region_pro_山西        0
addr_more_region_pro_广东        0
addr_more_region_pro_广西        0
addr_more_region_pro_新疆        0
addr_more_region_pro_江苏        0
addr_more_region_pro_江西        0
addr_more_region_pro_河北        0
addr_more_region_pro_河南        0
addr_more_region_pro_浙江        0
addr_more_region_pro_海南        0
addr_more_region_pro_港澳        0
addr_more_region_pro_湖北        0
addr_more_region_pro_湖南        0
addr_more_region_pro_甘肃        0
addr_more_region_pro_福建        0
addr_more_region_pro_西藏        0
addr_more_region_pro_贵州        0
          

,id,target,addr_more_region_pro_上海,addr_more_region_pro_云南,addr_more_region_pro_内蒙,addr_more_region_pro_北京,addr_more_region_pro_台湾,addr_more_region_pro_吉林,addr_more_region_pro_四川,addr_more_region_pro_天津,...,addr_more_region_city_黑龙江牡丹,addr_more_region_city_黑龙江绥化,addr_more_region_city_黑龙江鸡西,addr_more_region_city_黑龙江鹤岗,addr_more_region_city_黑龙江黑河,addr_more_region_city_黑龙江齐齐,addr_more_region_proportion,addr_more_Latitude,addr_more_longitude,addr_dis
0,501951980776722440,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,0.173209,0.696020,-0.478896
1,525890212484616200,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,0.173209,0.696020,-0.478896
2,599309364691472392,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-3.015684,1.005812,0.062246,0.704600
3,582559913071350024,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,0.573549,-1.207032,-0.478896
4,599017959926534408,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,0.485026,0.410799,-0.478896
5,575388917721731080,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,-0.886607,-1.589252,-0.478896
6,541887664329068552,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,0.173209,0.696020,-0.478896
7,605760099452981512,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,-1.652186,-0.319365,-0.478896
8,472286860756914440,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,0.173209,0.696020,-0.478896
9,600450091387457800,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,0.885369,-2.274449,-0.478896


In [18]:
fulldata.id = fulldata.id.astype("str")
mm = test11[["id","qq_bound_已绑定"]]
mm.id = mm.id.astype("str")
test = mm.merge(fulldata,how = 'left',on = 'id')
print(test.shape)
del test["qq_bound_已绑定"]
print(test.shape)
test

d:\gongju\python3.6 64\anzhuang\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


(47031, 525)
(47031, 524)


,id,addr_more_region_pro_上海,addr_more_region_pro_云南,addr_more_region_pro_内蒙,addr_more_region_pro_北京,addr_more_region_pro_台湾,addr_more_region_pro_吉林,addr_more_region_pro_四川,addr_more_region_pro_天津,addr_more_region_pro_宁夏,...,addr_more_region_city_黑龙江牡丹,addr_more_region_city_黑龙江绥化,addr_more_region_city_黑龙江鸡西,addr_more_region_city_黑龙江鹤岗,addr_more_region_city_黑龙江黑河,addr_more_region_city_黑龙江齐齐,addr_more_region_proportion,addr_more_Latitude,addr_more_longitude,addr_dis
0,637832512621515016,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,-1.168595,-0.961980,-0.478896
1,624385123294384392,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-0.923891,-1.514119,0.140252,0.850095
2,636912835502280712,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-2.388146,-0.931258,-1.648147,0.046710
3,638094054000693512,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,0.173209,0.696020,-0.478896
4,636732958538993928,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-1.656019,-1.455748,0.076942,0.944393
5,636668423459311624,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,0.173209,0.696020,-0.478896
6,628171292146667528,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-1.656019,-1.344945,-0.428726,2.071159
7,623732422298570760,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-1.656019,2.883437,1.601630,2.572899
8,637388003227996424,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,0.173209,0.696020,-0.478896
9,626505930430746888,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.540364,0.173209,0.696020,-0.478896


In [19]:
# cc = train[ff]
# cc.insert(0,"id",train.id)
# zz = train11.merge(cc,how = 'left',on = 'id')
# print(zz.shape,zz.isnull().sum().sort_values(ascending = False))
# zz.to_csv("../dealinput/train502ok.csv",index = False)
cc = test[ff]
cc.insert(0,"id",test.id)
cc.id = cc.id.astype("str")
test11.id = test11.id.astype("str")
zz = test11.merge(cc,how = 'left',on = 'id')
print(zz.shape,zz.isnull().sum().sort_values(ascending = False))
zz.to_csv("../dealinput/test502ok.csv",index = False)

d:\gongju\python3.6 64\anzhuang\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


(47031, 258) addr_dis                                 0
bank_name_more_cate_h                    0
phone_more_cate_top3_158                 0
phone_more_cate_top3_157                 0
phone_more_cate_top3_155                 0
phone_more_cate_top3_153                 0
phone_more_cate_top3_152                 0
phone_more_cate_top3_151                 0
phone_more_cate_top3_150                 0
phone_more_cate_top3_147                 0
phone_more_cate_top3_139                 0
phone_more_cate_top3_138                 0
phone_more_cate_top3_137                 0
phone_more_cate_top3_136                 0
phone_more_cate_top3_135                 0
phone_more_cate_top3_134                 0
phone_more_cate_top3_133                 0
phone_more_cate_top3_132                 0
phone_more_cate_top3_130                 0
phoneSame_1                              0
phoneSame_0                              0
havaCreditCard_1                         0
havaCreditCard_0                         

In [9]:
# trainx = train.iloc[:,2:]
trainx = train[ff]
trainy = train.iloc[:,1]
print("trainx.shape",trainx.shape)
print("trainy.shape",trainy.shape)
from sklearn.cross_validation import train_test_split
x_train,x_valid,y_train,y_valid = train_test_split(trainx,trainy,test_size = 0.2,random_state = 2)  
print("x_train.shape",x_train.shape)
print("x_valid.shape",x_valid.shape)
print("y_train.shape",y_train.shape)
print("y_valid.shape",y_valid.shape)

trainx.shape (120929, 69)
trainy.shape (120929,)
x_train.shape (96743, 69)
x_valid.shape (24186, 69)
y_train.shape (96743,)
y_valid.shape (24186,)


In [10]:
skf = StratifiedKFold(n_splits=5)
xgb1 = XGBClassifier(
#         booster = 'dart',
        learning_rate =0.1,
        n_estimators=5000,
        max_depth=5,
        min_child_weight=5,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=3,
        scale_pos_weight=1,
        StratifiedKFold = True,
        seed = 1)          
xgb_param = xgb1.get_xgb_params()
#构建稀疏矩阵，运行更快
xgtrain = xgb.DMatrix(x_train, label = y_train)
mm = xgb.cv(xgb_param, xgtrain, num_boost_round=1000, nfold=3,
    metrics='auc', early_stopping_rounds=50)
mm.iloc[-3:,:]  

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
46,0.560957,0.006208,0.643789,0.004670
47,0.562597,0.006765,0.644955,0.004324
48,0.563664,0.006594,0.645131,0.004603


In [11]:
import random
rand = []
for i in range(5):
    rand.append(random.uniform(0.75, 0.85))
rand,len(rand)

randSeed = []
for i in range(5):
    randSeed.append(random.randint(1, 66))

importance = 0
for i in range(5):
    xgb1 =  XGBClassifier(
        #         booster = 'dart',
                learning_rate =0.1,
                n_estimators=mm.shape[0],
                max_depth=5,
                min_child_weight=5,
                gamma=0.1,
                subsample=rand[i],
                colsample_bytree=0.8,
                objective= 'binary:logistic',
                nthread=3,
                scale_pos_weight=1,
                StratifiedKFold = True,          #0.866124	0.008005	0.965337	0.002286
                seed=randSeed[i])
    xgb1.fit(x_train,y_train)
    prexgb = xgb1.predict_proba(x_valid)[:,1]
    auc = roc_auc_score(y_valid,prexgb)
    print("xgb1 auc",auc)
    importance += xgb1.feature_importances_
importance = importance/5

xgb1 auc 0.5538597249124675
xgb1 auc 0.5478023390530524
xgb1 auc 0.5453386210513526
xgb1 auc 0.5445866018446415
xgb1 auc 0.5512628456281836


In [14]:
print(trainx.columns,importance)
number = 0
ff = []
for i in range(len(importance)):
    if importance[i] >0.001:
        ff.append(trainx.columns[i])
        number += 1
print(number)
ff,len(ff)

Index(['addr_more_region_pro_上海', 'addr_more_region_pro_云南',
       'addr_more_region_pro_北京', 'addr_more_region_pro_四川',
       'addr_more_region_pro_安徽', 'addr_more_region_pro_山东',
       'addr_more_region_pro_山西', 'addr_more_region_pro_广东',
       'addr_more_region_pro_广西', 'addr_more_region_pro_江苏',
       'addr_more_region_pro_江西', 'addr_more_region_pro_河北',
       'addr_more_region_pro_河南', 'addr_more_region_pro_浙江',
       'addr_more_region_pro_湖北', 'addr_more_region_pro_湖南',
       'addr_more_region_pro_福建', 'addr_more_region_pro_贵州',
       'addr_more_region_pro_辽宁', 'addr_more_region_pro_黑龙',
       'addr_more_region_diji_1', 'addr_more_region_diji_2',
       'addr_more_region_city_上海浦东新', 'addr_more_region_city_云南大理州',
       'addr_more_region_city_北京朝阳区', 'addr_more_region_city_吉林四平市',
       'addr_more_region_city_四川乐山市', 'addr_more_region_city_四川德阳市',
       'addr_more_region_city_四川成都市', 'addr_more_region_city_四川眉山市',
       'addr_more_region_city_四川雅安市', 'addr_more_regi

(['addr_more_region_pro_上海',
  'addr_more_region_pro_云南',
  'addr_more_region_pro_北京',
  'addr_more_region_pro_四川',
  'addr_more_region_pro_山东',
  'addr_more_region_pro_山西',
  'addr_more_region_pro_广东',
  'addr_more_region_pro_广西',
  'addr_more_region_pro_江苏',
  'addr_more_region_pro_江西',
  'addr_more_region_pro_河北',
  'addr_more_region_pro_河南',
  'addr_more_region_pro_浙江',
  'addr_more_region_pro_湖北',
  'addr_more_region_pro_湖南',
  'addr_more_region_pro_福建',
  'addr_more_region_pro_贵州',
  'addr_more_region_pro_辽宁',
  'addr_more_region_pro_黑龙',
  'addr_more_region_diji_1',
  'addr_more_region_diji_2',
  'addr_more_region_city_上海浦东新',
  'addr_more_region_city_云南大理州',
  'addr_more_region_city_北京朝阳区',
  'addr_more_region_city_吉林四平市',
  'addr_more_region_city_四川乐山市',
  'addr_more_region_city_四川德阳市',
  'addr_more_region_city_四川成都市',
  'addr_more_region_city_四川眉山市',
  'addr_more_region_city_四川雅安市',
  'addr_more_region_city_山西长治市',
  'addr_more_region_city_广东东莞市',
  'addr_more_region_city_广东佛